# 06 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to `Vertex AI` as an `Endpoint` for online prediction serving. The notebook covers the following steps:
1. Run the test steps locally.
2. Execute the model deployment `CI/CD` steps using `Cloud Build`.



## Setup

### Import libraries

In [1]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

### Setup Google Cloud project

In [2]:
PROJECT = 'grandelli-demo-295810' # Change to your project id.
REGION = 'us-central1' # Change to your region.

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]

print("Project ID:", PROJECT)
print("Region:", REGION)

Project ID: grandelli-demo-295810
Region: us-central1


### Set configurations

In [3]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

## 1. Run CI/CD steps locally

In [4]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

In [6]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_artifact -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.0, pluggy-1.0.0
rootdir: /home/jupyter/20220318_Training/gcp-partner-training-mlops
plugins: anyio-3.3.4
collecting ... 2022-03-15 18:15:45.755758: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
collected 1 item                                                               

src/tests/model_deployment_tests.py 2022-03-15 18:15:49.352236: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-15 18:15:49.352299: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed

### Run create endpoint

In [7]:
!python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

INFO:root:Creating endpoint chicago-taxi-tips-classifier
INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/155283586619/locations/us-central1/endpoints/2950605423834562560/operations/10041358660141056
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/155283586619/locations/us-central1/endpoints/2950605423834562560
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/155283586619/locations/us-central1/endpoints/2950605423834562560')
E0315 18:16:34.590918240      79 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0315 18:16:36.714152722      79 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
INFO:root:Endpoint is ready.
INFO:root:name: "projects/155283586619/locations/us-ce

### Run deploy model

In [8]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

INFO:root:serving resources: {'traffic_percentage': 100, 'machine_type': 'n1-standard-2', 'min_replica_count': 1, 'max_replica_count': 1, 'accelerator_type': None, 'accelerator_count': None}
INFO:root:Deploying model chicago-taxi-tips-classifier-v01 to endpoint chicago-taxi-tips-classifier
INFO:google.cloud.aiplatform.models:Deploying Model projects/155283586619/locations/us-central1/models/4079157352723382272 to Endpoint : projects/155283586619/locations/us-central1/endpoints/2950605423834562560
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/155283586619/locations/us-central1/endpoints/2950605423834562560/operations/8557873451409342464
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/155283586619/locations/us-central1/endpoints/2950605423834562560
INFO:root:Model is deployed.
INFO:root:None
INFO:root:None


### Test deployed model endpoint

In [9]:
!python -m pytest src/tests/model_deployment_tests.py::test_model_endpoint

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.0, pluggy-1.0.0
rootdir: /home/jupyter/20220318_Training/gcp-partner-training-mlops
plugins: anyio-3.3.4
collected 1 item                                                               

src/tests/model_deployment_tests.py .                                    [100%]

=============================== warnings summary ===============================
../../.local/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:22
  /home/jupyter/.local/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py:22: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
    import imp

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
======================== 1 passed, 1 warning in 15.17s =========================


## 2. Execute the Model Deployment CI/CD routine in Cloud Build

The CI/CD routine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in Vertex AI if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying model will be executed.

In [10]:
!echo $CICD_IMAGE_URI

gcr.io/grandelli-demo-295810/cicd:latest


In [11]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

Creating temporary tarball archive of 5 file(s) totalling 11.2 KiB before compression.
Uploading tarball of [build/.] to [gs://grandelli-demo-295810_cloudbuild/source/1647369123.326303-b762e62666924352843d30a9a7a0838b.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/grandelli-demo-295810/locations/global/builds/4fc316f4-3bcf-4fc1-9f1c-ca769288e49e].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/4fc316f4-3bcf-4fc1-9f1c-ca769288e49e?project=155283586619].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "4fc316f4-3bcf-4fc1-9f1c-ca769288e49e"

FETCHSOURCE
Fetching storage object: gs://grandelli-demo-295810_cloudbuild/source/1647369123.326303-b762e62666924352843d30a9a7a0838b.tgz#1647369123561907
Copying gs://grandelli-demo-295810_cloudbuild/source/1647369123.326303-b762e62666924352843d30a9a7a0838b.tgz#1647369123561907...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                
Ope

### Run CI/CD from model deployment using Cloud Build

In [ ]:
REPO_URL = "https://github.com/grandelli/gcp-partner-training-mlops.git" # Change to your github repo.
BRANCH = "main" 

In [ ]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m